In [ ]:
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense, Embedding
import numpy as np

In [ ]:
num_samples = 25000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'rus.txt'

In [ ]:
!head rus.txt

In [ ]:
#TODO

import string

START = '<START>'
END   = '<END>'

# Vectorize the data

# fill the following variables:
input_texts = []
target_texts = []
input_words = set()
target_words = set()


with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    input_text = input_text.translate(str.maketrans("", "", string.punctuation))
    target_text = target_text.translate(str.maketrans("", "", string.punctuation))
    
    
    # lowercase
    # append target text with aux tokens

In [ ]:
input_words = sorted(list(input_words))
target_words = sorted(list(target_words))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

In [ ]:
input_token_index = {word: i for i, word in enumerate(input_words)}
target_token_index = {word: i for i, word in enumerate(target_words)}

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length),    dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length),    dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),    dtype='float32')

In [ ]:
# TODO
# fill encoder_input_data, decoder_input_data, decoder_target_data

In [ ]:
embedding_dim = 100
latent_dim = 256  # Latent dimensionality of the encoding space.

In [ ]:
# Restore the model and construct the encoder and decoder.
model = load_model('word_seq2seq.h5')

encoder_inputs = model.input[0]   # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[4].output   # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]   # input_2
decoder_embed  = model.layers[3](decoder_inputs)
decoder_state_input_h = Input(shape=(latent_dim,), name='input_3')
decoder_state_input_c = Input(shape=(latent_dim,), name='input_4')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[5]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_embed, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[6]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [ ]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_word_index = {i: word for word, i in input_token_index.items()}
reverse_target_word_index = {i: word for word, i in target_token_index.items()}

In [ ]:
# TODO

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = 

    # Generate empty target sequence of length 1.
    # Populate the first character of target sequence with the start character.
    target_seq = 

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        
        # calculate probabilties for next token
        # TODO
        next_char_prob = 
        sampled_token = 
        decoded_sentence.append(sampled_token)

        # Exit condition: either hit max length
        # or find stop token.
        # TODO

        # Update the target sequence (of length 1).
        # TODO

        # Update states
        # TODO

    return decoded_sentence

In [ ]:
from nltk.translate.bleu_score import sentence_bleu


scores = []

for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    decoded_sentence[:-1]
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    
    s = sentence_bleu(target_texts[seq_index], decoded_sentence, weights=(0.25, 0.25, 0.25, 0.25))
    scores.append(s)
    
print('mean {} std {}'.format(np.mean(scores), np.std(scores)))